In [10]:
import asn1
import hashlib
from random import randint
from pygost.gost34112012256 import GOST34112012256 as gost
from pygost import gost34112012
from sys import getdefaultencoding
def to_asn(xQ,yQ,p,A,B,xP,yP,q,r,s):
    my_string = 'gostSignKey'
    getdefaultencoding() 
    type(my_string) 
    gost_string=my_string.encode()
    encoder = asn1.Encoder()
    encoder.start()

    encoder.enter(asn1.Numbers.Sequence) 
    encoder.enter(asn1.Numbers.Set) 
    encoder.enter(asn1.Numbers.Sequence) 
    encoder.write(b'\x80\x06\x07\x00', asn1.Numbers.OctetString)  
    encoder.write(gost_string, asn1.Numbers.UTF8String)  
    encoder.enter(asn1.Numbers.Sequence) 
    encoder.write(xQ,asn1.Numbers.Integer)
    encoder.write(yQ,asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence) 
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(p,asn1. Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(A, asn1.Numbers.Integer)
    encoder.write(B, asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(xP,asn1.Numbers.Integer)
    encoder.write(yP,asn1.Numbers.Integer)
    encoder.leave()
    encoder.write(q,asn1.Numbers.Integer)
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(r,asn1.Numbers.Integer)
    encoder.write(s,asn1.Numbers.Integer)
    encoder.leave()
    encoder.leave()
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.leave()
    encoder.leave()
    return encoder.output()

def Sign(E,P,file,d,Q,m,A,B,p):
        q=m
        with open(file, "rb") as file:
            data = file.read()
        #print(data)
        hash = gost(data).digest() #step 2
        e = int(gost34112012.GOST34112012(data).hexdigest(), 16) % q
        print("e:",e)
        if e==0:
            e=1
        while 1:
            k = randrange(1, q) 
            C=k*P
            xC=C[0]
            r=mod(xC, q)
            if r==0:
                continue
            s=mod(r*d+k*e,q)
            if s==0:
                continue
            break
        print("C:",C)
        print("p:",p)
        print("XQ:",Q[0])
        print("yQ:",Q[1])
        print("A:",A)
        print("B:",B)
        print("xP:",P[0])
        print("yP:",P[1])
        print("q:",q)
        print("r:",r)
        print("s:",s)
        sign=to_asn(int(Q[0]),int(Q[1]),p,A,B,int(P[0]),int(P[1]),q,int(r),int(s))
        with open("sign_test", "wb") as file:
            file.write(sign)
                
                
        








a = 1
b = 41431894589448105498289586872587560387979247722721848579560344157562082667257
p = 57896044622894643241131754937450315750132642216230685504884320870273678881443
m = 28948022311447321620565877468725157875067316353637126186229732812867492750347
x = 54672615043105947691210796380713598019547553171137275980323095812145568854782
y = 42098178416750523198643432544018510845496542305814546233883323764837032783338


K=GF(p)
E=EllipticCurve(K,[a,b])
P=E(x,y)


d=randrange(1,m)
Q=d*P
file='test.txt'
Sign(E,P,file,d,Q,m,a,b,p)

P: 57896044622894643241131754937450315750134632707274252372459465625734985500694
P1: (0 : 1 : 0)
e: 28037270501518147842175774393183664493402184281856724416360318565814580737840
C: (34862612851012070140148973237705129060221136749232330724340572786639637699974 : 21519769150205854994789841395345337371163831199403251033815386720763419133844 : 1)
p: 57896044622894643241131754937450315750132642216230685504884320870273678881443
XQ: 21623636645870790334869860875015014859341981221828646349597702513817563862931
yQ: 888316329594168405538181878042071549074391152693231855376726446926755206863
A: 1
B: 41431894589448105498289586872587560387979247722721848579560344157562082667257
xP: 54672615043105947691210796380713598019547553171137275980323095812145568854782
yP: 42098178416750523198643432544018510845496542305814546233883323764837032783338
q: 28948022311447321620565877468725157875067316353637126186229732812867492750347
r: 5914590539564748519583095768979971185153820395595204538110839973772144949627
s

In [1]:
import asn1
import hashlib
from random import randint
from pygost import gost34112012
from pygost.gost34112012256 import GOST34112012256 as gost
from sys import getdefaultencoding


def parse(decoder, integers): # такой же парсер, как и в шифровании
    while not decoder.eof():
        try:
            tag = decoder.peek()

            if tag.nr == asn1.Numbers.Null:
                break

            if tag.typ == asn1.Types.Primitive:
                tag, value = decoder.read()

                if tag.nr == asn1.Numbers.Integer:
                    integers.append(value)

            else:
                decoder.enter()
                integers = parse(decoder, integers)
                decoder.leave()

        except asn1.Error:
            break

    return integers
def from_asn(filename): 
    integers = []
    with open(filename, "rb") as file:
        data = file.read()
    decoder = asn1.Decoder()
    decoder.start(data)
    integers = parse(decoder, integers) 
    return integers[0],integers[1],integers[2],integers[3],integers[4],integers[5],integers[6],integers[7],integers[8],integers[9]



def check_sign(xQ,yQ,p,A,B,xP,yP,q,r,s,filename):
    if (r<0 or r > q) and (s<0 or s>q):
        return "Wrong sign"
    with open(filename, "rb") as file:
        data = file.read()
    print(data)
    K=GF(p)
    E=EllipticCurve(K,[A,B])
    print("XQ:",xQ)
    print("yQ:",yQ)
    Q=E(xQ,yQ)
    P=E(xP,yP)
    e = int(gost34112012.GOST34112012(data).hexdigest(), 16) % q
    if e==0:
        e=1
    print("e:",e)
    v = inverse_mod(e, q)
    z1=int(mod(s*v,q))
    z2=int(mod(-r*v,q))
    #z1,z2=int(z1),int(z2)
    C=z1*P+z2*Q
    print("C:",C)
    R=mod(C[0],q)
    print(r)
    print(R)
    if R==r:
        return "Sign correct"
    else:
        return "Wrong sign"
    
    
file='sign_test'
filename='test.txt'
xQ,yQ,p,A,B,xP,yP,q,r,s=from_asn(file)
print("p:",p)
print("XQ:",xQ)
print("yQ:",yQ)
print("A:",A)
print("B:",B)
print("xP:",xP)
print("yP:",yP)
print("q:",q)
print("r:",r)
print("s:",s)
check_sign=check_sign(xQ,yQ,p,A,B,xP,yP,q,r,s,filename)
print(check_sign)


p: 57896044622894643241131754937450315750132642216230685504884320870273678881443
XQ: 24300937468070635862117133599262990320720567267512515732054200308986590205644
yQ: 5375249113777024673345376487510638772355913623998059523602376315037383722166
A: 1
B: 41431894589448105498289586872587560387979247722721848579560344157562082667257
xP: 54672615043105947691210796380713598019547553171137275980323095812145568854782
yP: 42098178416750523198643432544018510845496542305814546233883323764837032783338
q: 28948022311447321620565877468725157875067316353637126186229732812867492750347
r: 19926424065721960338409598840813730526390310353493177192977439681480604982680
s: 2290144830526078456924014866636074425494287158771277634908226752784772825536
b'1221312312312321'
XQ: 24300937468070635862117133599262990320720567267512515732054200308986590205644
yQ: 5375249113777024673345376487510638772355913623998059523602376315037383722166
e: 25465322168240033691092120070702724141905813878862661680238325885821185264841
